# Классификация комментариев с использованием BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. 

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Установим-необходимые-библиотеки" data-toc-modified-id="Установим-необходимые-библиотеки-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Установим необходимые библиотеки</a></span></li><li><span><a href="#Импортируем-необходимые-библиотеки" data-toc-modified-id="Импортируем-необходимые-библиотеки-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Импортируем необходимые библиотеки</a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Проверка-данных" data-toc-modified-id="Проверка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Проверка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных-для-обучения" data-toc-modified-id="Подготовка-данных-для-обучения-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовка данных для обучения</a></span></li><li><span><a href="#Инициализация-и-настройка-модели" data-toc-modified-id="Инициализация-и-настройка-модели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Инициализация и настройка модели</a></span></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обучение модели</a></span></li><li><span><a href="#Оценка-модели" data-toc-modified-id="Оценка-модели-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Оценка модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

### Установим необходимые библиотеки

In [1]:
#!pip install transformers -q
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu124
#!pip install scikit-learn -q
#!pip install tqdm -q
!pip install numpy==1.24.0 -q

### Импортируем необходимые библиотеки

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivanm\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ivanm\AppData\Roaming\nltk_data...


True

In [3]:
# задаём значение констант
TEST_SIZE = 0.2
RANDOM_STATE = 42 

# игнорируем предупреждения
import warnings
warnings.simplefilter("ignore")

## Подготовка

### Загрузка данных

Считаем CSV-файл в датафрейм

In [4]:
local_file_path = '/datasets/toxic_comments.csv'
online_file_path = 'https://xxx/toxic_comments.csv'

try:
    if os.path.exists(local_file_path):
        df = pd.read_csv(local_file_path)
        print("Данные загружены из локальной директории")
    else:
        df = pd.read_csv(online_file_path)
        print("Данные загружены из сети")
except Exception as e:
    print(f"Произошла ошибка при чтении файла: {e}")

Данные загружены из сети


### Проверка данных

Проверим структуру данных, наличие пропусков и баланс классов для лучшего понимания данных

Выведем общую информацию о датафрейме

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Выведем количество пропусков

In [6]:
print('Количество пропусков:\n', df.isna().sum())

Количество пропусков:
 Unnamed: 0    0
text          0
toxic         0
dtype: int64


Выведеем первые строки датафрейма

In [7]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Проверим баланс классов

In [8]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

**Вывод:**

Общая информация о датафрейме:
- Число строк: 159292
- Пропущенные значения: отсутствуют

Наблюдается дисбаланс классов:
- Не токсичные комментарии: 89.84%
- Токсичные комментарии: 10.16%

## Обучение

### Подготовка данных для обучения

Подготовим данные для обучения и тестирования, включая токенизацию текста и создание DataLoader

Проведём очистку и лемматизацию данных

Напишем функцию для очистки данных

In [9]:
def clean_text(text):
    # Приводим текст к нижнему регистру
    text = text.lower()
    # Удаляем HTML теги
    text = re.sub(r'<.*?>', '', text)
    # Удаляем URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Удаляем специальные символы и цифры
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    # Удаляем лишние пробелы
    text = text.strip()
    return text

Напишем функцию для лемматизации данных

In [10]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = text.split()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

Применим эти функции

In [11]:
df['cleaned_text'] = df['text'].apply(clean_text).apply(lemmatize_text)

Выделим признак и целевую переменную

In [12]:
X = df['cleaned_text']
y = df['toxic']

Разделим данные на обучающую и тестовую выборки

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

Инициализируем токенизатор BERT

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Напишем функцию для тоокенизации текста

In [15]:
def tokenize_text(texts, tokenizer, max_len=512):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True  
        )

        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

Токенизируем тренировочные и тестовые данные

In [16]:
train_inputs, train_masks = tokenize_text(X_train, tokenizer)
test_inputs, test_masks = tokenize_text(X_test, tokenizer)

Преобразуем целевыее метки в тензоры

In [17]:
train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)

Создадим DataLoader для обучения и тестирования

In [18]:
batch_size = 16

In [19]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [20]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

### Инициализация и настройка модели

Инициализируем модель, установим оптимизатор и настроим устройство для вычислений

Инициалиируем модели BERT для классификации

In [21]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Настроим оптимизатор

In [22]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [23]:
# Настройка устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [24]:
# Проверка доступности CUDA
print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Используемый GPU: {torch.cuda.get_device_name(0)}")

CUDA доступна: True
Используемый GPU: NVIDIA GeForce RTX 3050


### Обучение модели

Напишем функцию для обучения модели

In [25]:
def train_model(model, dataloader, optimizer, device, epochs=4):
    model.train()

    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        epoch_iterator = tqdm(dataloader, desc="Training")
        for step, batch in enumerate(epoch_iterator):
            batch_inputs, batch_masks, batch_labels = tuple(t.to(device) for t in batch)
            model.zero_grad()

            outputs = model(input_ids=batch_inputs, attention_mask=batch_masks, labels=batch_labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1} loss: {loss.item()}")

Обучим модель

In [26]:
train_model(model, train_dataloader, optimizer, device)

Epoch 1/4


Training:   0%|          | 0/7965 [00:00<?, ?it/s]

Epoch 1 loss: 0.3561801314353943
Epoch 2/4


Training:   0%|          | 0/7965 [00:00<?, ?it/s]

Epoch 2 loss: 0.005918531212955713
Epoch 3/4


Training:   0%|          | 0/7965 [00:00<?, ?it/s]

Epoch 3 loss: 4.3775307858595625e-05
Epoch 4/4


Training:   0%|          | 0/7965 [00:00<?, ?it/s]

Epoch 4 loss: 0.015288498252630234


### Оценка модели

Оценим модель на тестовых данных и рассчитаем метрику F1

Напишем фуннкцию для оценки модели

In [27]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []

    for batch in tqdm(dataloader, desc="Evaluating"):
        batch_inputs, batch_masks, batch_labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            outputs = model(input_ids=batch_inputs, attention_mask=batch_masks)

        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(batch_labels.cpu().numpy())

    return predictions, true_labels

Оценим модель

In [28]:
y_pred, y_true = evaluate_model(model, test_dataloader, device)

Evaluating:   0%|          | 0/1992 [00:00<?, ?it/s]

In [29]:
# Оценка метрики F1
f1 = f1_score(y_true, y_pred)
print("F1 Score:", f1)

F1 Score: 0.8338136407300673


## Выводы

- Наша модель на основе BERT успешно классифицирует токсичные и не токсичные комментарии.
- Достигнутая метрика F1 составляет 0.8338, что превышает требуемый порог в 0.75.
- Проект продемонстрировал эффективное применение трансформеров и BERT для задачи классификации текста.